In [3]:
import optuna
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# === Load your split data ===
base_path = Path("/Users/tusharsingh/Work/Project/tcga-mldl/results/data/data_splits/15pct_70")
X_train = pd.read_csv(base_path / "X_train.csv", index_col=0)
X_test = pd.read_csv(base_path / "X_test.csv", index_col=0)
y_train = pd.read_csv(base_path / "y_train.csv", index_col=0).squeeze()
y_test = pd.read_csv(base_path / "y_test.csv", index_col=0).squeeze()

# === Split test into validation and test ===
X_val, X_final_test, y_val, y_final_test = train_test_split(
    X_test, y_test, test_size=0.5, stratify=y_test, random_state=42
)

X_train_np = X_train.values
y_train_np = y_train.values
X_val_np = X_val.values
y_val_np = y_val.values

# === Build CNN model with trial params ===
def build_model(trial):
    model = Sequential([
        Input(shape=(X_train_np.shape[1], 1)),
        Conv1D(
            filters=trial.suggest_categorical("filters", [32, 64, 128]),
            kernel_size=trial.suggest_categorical("kernel_size", [2, 8, 16]),
            strides=trial.suggest_categorical("stride", [1]),
            activation='relu'
        ),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(trial.suggest_categorical("dense_units", [64,128]), activation='relu'),
        Dropout(0.07),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# === Objective for tuning ===
def objective(trial):
    model = build_model(trial)
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=0)
    model.fit(
        X_train_np[..., np.newaxis], y_train_np,
        validation_data=(X_val_np[..., np.newaxis], y_val_np),
        epochs=30, batch_size=64, verbose=0,
        callbacks=[early_stop]
    )
    y_pred = (model.predict(X_val_np[..., np.newaxis]) > 0.5).astype(int).flatten()
    return accuracy_score(y_val_np, y_pred)

# === Run study ===
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best Params:", study.best_params)
print("Best Val Accuracy:", study.best_value)


[I 2025-08-05 10:30:15,537] A new study created in memory with name: no-name-42ad83fa-5683-430f-8d70-1d0636548384


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


[I 2025-08-05 10:30:20,499] Trial 0 finished with value: 0.6447368421052632 and parameters: {'filters': 64, 'kernel_size': 16, 'stride': 1, 'dense_units': 64}. Best is trial 0 with value: 0.6447368421052632.


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


[I 2025-08-05 10:30:35,523] Trial 1 finished with value: 0.6973684210526315 and parameters: {'filters': 128, 'kernel_size': 8, 'stride': 1, 'dense_units': 64}. Best is trial 1 with value: 0.6973684210526315.


...

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


[I 2025-08-05 10:48:56,792] Trial 48 finished with value: 0.7105263157894737 and parameters: {'filters': 64, 'kernel_size': 2, 'stride': 1, 'dense_units': 128}. Best is trial 23 with value: 0.7631578947368421.


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step


[I 2025-08-05 10:49:29,381] Trial 49 finished with value: 0.6842105263157895 and parameters: {'filters': 128, 'kernel_size': 16, 'stride': 1, 'dense_units': 128}. Best is trial 23 with value: 0.7631578947368421.


Best Params: {'filters': 64, 'kernel_size': 2, 'stride': 1, 'dense_units': 128}
Best Val Accuracy: 0.7631578947368421
